# 04 - Storytelling y Presentación de Resultados
## Proyecto Final: Análisis de Datos
### Objetivo: Sintetizar hallazgos y crear visualizaciones para presentación

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para presentación
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

print('Librerías cargadas para storytelling')

Librerías cargadas para storytelling


In [2]:
# Cargar todos los datos necesarios
clients = pd.read_csv('../data/clients_curated.csv')
projects = pd.read_csv('../data/projects_curated.csv')

# Convertir fechas
date_cols = ['start_date', 'planned_end_date', 'actual_end_date']
for col in date_cols:
    projects[col] = pd.to_datetime(projects[col])

print('Datos cargados para análisis final')

Datos cargados para análisis final


### 1. Resumen Ejecutivo de Hallazgos

In [3]:
print('=== RESUMEN EJECUTIVO ===')
print(f'• Total de clientes analizados: {len(clients)}')
print(f'• Total de proyectos completados: {len(projects[projects["status"] == "completed"])}')
print(f'• Tasa promedio de satisfacción: {clients["satisfaction_score"].mean():.2f}/5')
print(f'• Presupuesto promedio por proyecto: ${projects["budget_usd"].mean():,.2f}')
print(f'• Tasa de renovación de contratos: {clients["renewed_contract"].mean()*100:.1f}%')

=== RESUMEN EJECUTIVO ===
• Total de clientes analizados: 100
• Total de proyectos completados: 0
• Tasa promedio de satisfacción: 3.03/5
• Presupuesto promedio por proyecto: $19,838.55
• Tasa de renovación de contratos: 38.0%
